## More Lions is All You Need

running scoring and voting using `llama3` model via `ollama` service

In [1]:
system = "you are a helpful assistant."
instruct = "think step by step, concisely yet comprehensively explain the balance sheet. return around 100 words"

The goal is to find the best answer from 5 generations of the same questions

In [2]:
import lionagi as li

In [3]:
durations = []

async def explain():
    
    imodel = li.iModel(model="llama3", provider="ollama")
    branch = li.Branch(system=system, imodel=imodel)
    
    await branch.chat(instruction=instruct)
    return branch

In [4]:
# write reports

reports = []
for i in range(5):
    writer = await explain()
    report = writer.last_response.response
    reports.append(report)
    print(f"Report No.{i+1} finished.")

Report No.1 finished.
Report No.2 finished.
Report No.3 finished.
Report No.4 finished.
Report No.5 finished.


### Get a group of critics

In [5]:
system = """
    as a highly critical professor, score an answer, only return a 
    positive integer
"""

instruction = """
    meticulously and critically give an integer score between 0-10 on the 
    following explaination of balance sheet.
"""

In [6]:
import numpy as np
from lionagi.libs import to_num

async def score(context):
    imodel = li.iModel(model="llama3", provider="ollama")
    branch = li.Branch(system=system, imodel=imodel)
    out_ = await branch.chat(instruction=instruction, context=context)
    
    # let's convert the output to a number
    try:
        out_ = to_num(out_)
    except:
        out_ = -1
    return out_

In [7]:
async def group_score(context):
    outs = []
    for _ in range(5):
        out_ = await score(context)
        outs.append(out_)
    
    outs = [i for i in outs if i != -1]
    return np.mean(outs)

In [8]:
scores = []

# run scoring
for idx, i in enumerate(reports):
    score_ = await group_score(i)    
    scores.append((score_, i))
    print(f"Report: No.{idx + 1}, Score: {score_}")

Report: No.1, Score: 7.6
Report: No.2, Score: 7.0
Report: No.3, Score: 7.6
Report: No.4, Score: 5.4
Report: No.5, Score: 7.4


### Reports

In [9]:
# rank scores
scores = sorted(scores, key=lambda x: x[0], reverse=True)

In [10]:
from IPython.display import display, Markdown

In [11]:
score, report = scores[0]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report}") )

Score: 7.6


**Report:**

Here's a concise and comprehensive explanation of the balance sheet:

The Balance Sheet is a financial statement that presents a company's financial position at a specific point in time. It's divided into three main sections: Assets, Liabilities, and Equity.

**Assets**: What the company owns (e.g., cash, inventory, property).

* Current assets: Expected to be converted or used within 1 year (e.g., accounts receivable).
* Non-current assets: Long-term investments (e.g., property, equipment).

**Liabilities**: What the company owes to others (e.g., loans, accounts payable).

* Current liabilities: Expected to be paid within 1 year (e.g., taxes owed).
* Non-current liabilities: Longer-term debts (e.g., long-term loans).

**Equity**: The company's net worth, representing shareholders' stake.

By subtracting Liabilities from Assets, the Balance Sheet shows the company's Net Worth or Shareholders' Equity. This fundamental financial statement provides a snapshot of a company's financial health!

In [12]:
score, report = scores[1]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report}") )

Score: 7.6


**Report:**

Here's a concise explanation of a balance sheet:

A balance sheet is a snapshot of a company's financial position at a specific point in time. It lists its assets (what it owns), liabilities (what it owes), and equity (ownership interest). Assets are categorized into current (easily convertible to cash) and non-current (long-term investments). Liabilities include short-term debts and long-term obligations. Equity is comprised of common stock, retained earnings, and other capital accounts. The balance sheet equation: Assets = Liabilities + Equity ensures that the company's value is accurately represented.

In [13]:
score, report = scores[2]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report}") )

Score: 7.4


**Report:**

Here's a concise explanation of a balance sheet:

**What is it?**
A balance sheet is a financial statement that shows a company's financial position at a specific point in time.

**Key components:**

1. **Assets**: What a company owns, such as cash, accounts receivable, inventory, property, and equipment.
2. **Liabilities**: What a company owes, including accounts payable, loans, and taxes owed.
3. **Equity**: The company's net worth, representing the amount of ownership interest.

**Balance:**
The total value of assets equals the total value of liabilities plus equity. This ensures the statement is "in balance."

**Why important?**
A balance sheet helps investors, creditors, and management understand a company's financial health, liquidity, and solvency. It's essential for making informed business decisions.

In [14]:
score, report = scores[3]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report}") )

Score: 7.0


**Report:**

Here's a concise explanation of a balance sheet:

A balance sheet is a financial statement that shows a company's financial position at a specific point in time. It's divided into three main sections: Assets, Liabilities, and Equity.

**Assets**: What the company owns or has control over (e.g., cash, accounts receivable, inventory, property).

**Liabilities**: What the company owes to others (e.g., accounts payable, loans, taxes owed).

**Equity**: The ownership interest in the business (e.g., common stock, retained earnings). The total value of assets must equal the total value of liabilities and equity. This ensures the balance sheet is, well, balanced!

In [15]:
score, report = scores[4]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report}") )

Score: 5.4


**Report:**

Here's a concise explanation of a balance sheet:

**What is it?**
A balance sheet is a financial statement that presents a company's financial position at a specific point in time.

**Components:**

1. **Assets**: What the company owns (e.g., cash, accounts receivable, inventory, property).
2. **Liabilities**: What the company owes to others (e.g., accounts payable, loans, taxes).
3. **Equity**: The company's net worth, equal to assets minus liabilities.

**Relationship:**
Assets = Liabilities + Equity

In essence, a balance sheet shows what a company has (assets), what it owes (liabilities), and what remains after settling debts (equity).